In [2]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [4]:
databasePath = "'../eICU/training/"
exportPath = "'../eICU/training/"

#### Interpolating Data across patient timeseries

In [3]:
finalMerge = pd.read_csv(databasePath + 'finalMerge.csv')
finalMerge = finalMerge.sort_values(by=['patientunitstayid', 'observationoffset'])
finalMerge

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
40960384,141227,-1893.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.433090,NaN
40960386,141227,-1773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.433090,NaN
40960385,141227,-1663.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.433090,NaN
40657508,141227,-1566.0,NaN,NaN,NaN,NaN,1.40,48.2,NaN,NaN,NaN
40960387,141227,-1351.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.433090,NaN
...,...,...,...,...,...,...,...,...,...,...,...
41756721,3353254,5326.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.767580,NaN
41756711,3353254,5491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.383790,NaN
40960381,3353254,5558.0,NaN,NaN,NaN,NaN,1.67,11.7,NaN,NaN,NaN
41756719,3353254,5926.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.151371,NaN


In [4]:
print("temperature: ", len(finalMerge['temperature'].unique()))
print("heartrate: ", len(finalMerge['heartrate'].unique()))
print("creatinine: ", len(finalMerge['creatinine'].unique()))
print("wbcx1000: ", len(finalMerge['wbcx1000'].unique()))
print("urineoutputbyweight: ", len(finalMerge['urineoutputbyweight'].unique()))
print("diagnosis: ", len(finalMerge['diagnosis'].unique()))

temperature:  1946
heartrate:  271
creatinine:  3707
wbcx1000:  4489
urineoutputbyweight:  102159
diagnosis:  3


In [8]:
#finalMerge_small = finalMerge[finalMerge.patientunitstayid.isin([key for key, val in items if (val < 1000)])]

In [9]:
finalMerge.diagnosis.value_counts()

0.0    151121
1.0     82935
Name: diagnosis, dtype: int64

In [10]:
idList = sorted(set(list(finalMerge['patientunitstayid'])))

39669 20000 19669


In [9]:
cols = ['temperature', 'heartrate', 'respiration', 'systemicsystolic', 'creatinine', 'wbcx1000', 'lactate', 'urineoutputbyweight']
interpolatedData = pd.DataFrame()

In [24]:
x = 0

for id in idList:
    df = finalMerge[finalMerge['patientunitstayid'] == id]
    countNull = df[cols].isnull().sum().sum()
    percentOfNull = (countNull / (len(df.index) * 8)) * 100
    # print(percentOfNull)
    
    if x%1000 == 0:
        print(x)
    x+=1

    if percentOfNull < 80:
        # Interpolating Patient Data based on closet value
        df.loc[:, 'temperature'] = df['temperature'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'heartrate'] = df['heartrate'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'respiration'] = df['respiration'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'systemicsystolic'] = df['systemicsystolic'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'creatinine'] = df['creatinine'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'wbcx1000'] = df['wbcx1000'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'lactate'] = df['lactate'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'urineoutputbyweight'] = df['urineoutputbyweight'].interpolate(method='linear', limit_direction='both')
        finalData = interpolatedData.append(df)

interpolatedData

0


/home/sanjay/.local/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
40960384,141227,-1893.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,NaN
40960386,141227,-1773.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,NaN
40960385,141227,-1663.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,NaN
40657508,141227,-1566.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,NaN
40960387,141227,-1351.0,NaN,112.0,49.0,158.0,1.40,47.95,4.3,2.433090,NaN
...,...,...,...,...,...,...,...,...,...,...,...
41285073,1579769,17268.0,NaN,73.0,17.0,NaN,0.84,8.09,0.8,6.641209,NaN
41285077,1579769,17448.0,NaN,73.0,17.0,NaN,0.84,8.09,0.8,1.660302,NaN
41285072,1579769,17868.0,NaN,73.0,17.0,NaN,0.84,8.09,0.8,4.980907,NaN
41285053,1579769,19188.0,NaN,73.0,17.0,NaN,0.84,8.09,0.8,3.735680,NaN


In [15]:
interpolatedData.to_csv(exportPath + "interpolatedData.csv", sep=',', index=False, encoding='utf-8')

In [ ]:
%reset -f

import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [5]:
finalData = pd.read_csv(databasePath + '/interpolatedData.csv')
finalData

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
0,141227,-1893.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,NaN
1,141227,-1773.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,NaN
2,141227,-1663.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,NaN
3,141227,-1566.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,NaN
4,141227,-1351.0,NaN,112.0,49.0,158.0,1.40,47.95,4.3,2.433090,NaN
...,...,...,...,...,...,...,...,...,...,...,...
38721752,3353254,5326.0,NaN,82.0,12.0,NaN,1.65,11.32,NaN,4.767580,NaN
38721753,3353254,5491.0,NaN,82.0,12.0,NaN,1.66,11.51,NaN,2.383790,NaN
38721754,3353254,5558.0,NaN,82.0,12.0,NaN,1.67,11.70,NaN,4.767580,NaN
38721755,3353254,5926.0,NaN,82.0,12.0,NaN,1.67,11.70,NaN,7.151371,NaN


In [10]:
finalData[cols].isnull().sum().sum()

79406566

In [ ]:
finalData['diagnosis'] = finalData['diagnosis'].fillna(0)
finalData.diagnosis.value_counts()

In [0]:
sepsisOnly = finalData[finalData['diagnosis'] == 1]
sepsisOnly

In [ ]:
sepsisOnly['temperature'] = sepsisOnly['temperature'].fillna(sepsisOnly['temperature'].mean())
sepsisOnly['heartrate'] = sepsisOnly['heartrate'].fillna(sepsisOnly['heartrate'].mean())
sepsisOnly['respiration'] = sepsisOnly['respiration'].fillna(sepsisOnly['respiration'].mean())
sepsisOnly['systemicsystolic'] = sepsisOnly['systemicsystolic'].fillna(sepsisOnly['systemicsystolic'].mean())
sepsisOnly['creatinine'] = sepsisOnly['creatinine'].fillna(sepsisOnly['creatinine'].mean())
sepsisOnly['wbcx1000'] = sepsisOnly['wbcx1000'].fillna(sepsisOnly['wbcx1000'].mean())
sepsisOnly['lactate'] = sepsisOnly['lactate'].fillna(sepsisOnly['lactate'].mean())
sepsisOnly['urineoutputbyweight'] = sepsisOnly['urineoutputbyweight'].fillna(sepsisOnly['urineoutputbyweight'].mean())

print(sepsisOnly[cols].isnull().sum().sum())

In [ ]:
notSepsis = finalData[finalData['diagnosis'] == 0]
notSepsis

In [ ]:
notSepsis['temperature'] = notSepsis['temperature'].fillna(notSepsis['temperature'].mean())
notSepsis['heartrate'] = notSepsis['heartrate'].fillna(notSepsis['heartrate'].mean())
notSepsis['respiration'] = notSepsis['respiration'].fillna(notSepsis['respiration'].mean())
notSepsis['systemicsystolic'] = notSepsis['systemicsystolic'].fillna(notSepsis['systemicsystolic'].mean())
notSepsis['creatinine'] = notSepsis['creatinine'].fillna(notSepsis['creatinine'].mean())
notSepsis['wbcx1000'] = notSepsis['wbcx1000'].fillna(notSepsis['wbcx1000'].mean())
notSepsis['lactate'] = notSepsis['lactate'].fillna(notSepsis['lactate'].mean())
notSepsis['urineoutputbyweight'] = notSepsis['urineoutputbyweight'].fillna(notSepsis['urineoutputbyweight'].mean())

print(notSepsis[cols].isnull().sum().sum())

In [ ]:
finalData = notSepsis.append(sepsisOnly)
del sepsisOnly, notSepsis

finalData = finalData.astype({'diagnosis': int})
finalData = finalData.sort_values(['patientunitstayid','observationoffset'])
finalData

In [ ]:
finalData.to_csv(exportPath + "finalData.csv", sep=',', index=False, encoding='utf-8')

In [12]:
sepsisOnly['temperature'] = sepsisOnly['temperature'].fillna(sepsisOnly['temperature'].mean())
sepsisOnly['heartrate'] = sepsisOnly['heartrate'].fillna(sepsisOnly['heartrate'].mean())
sepsisOnly['respiration'] = sepsisOnly['respiration'].fillna(sepsisOnly['respiration'].mean())
sepsisOnly['systemicsystolic'] = sepsisOnly['systemicsystolic'].fillna(sepsisOnly['systemicsystolic'].mean())
sepsisOnly['creatinine'] = sepsisOnly['creatinine'].fillna(sepsisOnly['creatinine'].mean())
sepsisOnly['wbcx1000'] = sepsisOnly['wbcx1000'].fillna(sepsisOnly['wbcx1000'].mean())
sepsisOnly['lactate'] = sepsisOnly['lactate'].fillna(sepsisOnly['lactate'].mean())
sepsisOnly['urineoutputbyweight'] = sepsisOnly['urineoutputbyweight'].fillna(sepsisOnly['urineoutputbyweight'].mean())
sepsisOnly[cols].isnull().sum().sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

0

In [13]:
notSepsis = finalData[finalData['diagnosis'] == 0]
notSepsis

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
0,141227,-1893.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,0.0
1,141227,-1773.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,0.0
2,141227,-1663.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,0.0
3,141227,-1566.0,NaN,112.0,49.0,158.0,1.40,48.20,4.3,2.433090,0.0
4,141227,-1351.0,NaN,112.0,49.0,158.0,1.40,47.95,4.3,2.433090,0.0
...,...,...,...,...,...,...,...,...,...,...,...
38721752,3353254,5326.0,NaN,82.0,12.0,NaN,1.65,11.32,NaN,4.767580,0.0
38721753,3353254,5491.0,NaN,82.0,12.0,NaN,1.66,11.51,NaN,2.383790,0.0
38721754,3353254,5558.0,NaN,82.0,12.0,NaN,1.67,11.70,NaN,4.767580,0.0
38721755,3353254,5926.0,NaN,82.0,12.0,NaN,1.67,11.70,NaN,7.151371,0.0


In [14]:
notSepsis['temperature'] = notSepsis['temperature'].fillna(notSepsis['temperature'].mean())
notSepsis['heartrate'] = notSepsis['heartrate'].fillna(notSepsis['heartrate'].mean())
notSepsis['respiration'] = notSepsis['respiration'].fillna(notSepsis['respiration'].mean())
notSepsis['systemicsystolic'] = notSepsis['systemicsystolic'].fillna(notSepsis['systemicsystolic'].mean())
notSepsis['creatinine'] = notSepsis['creatinine'].fillna(notSepsis['creatinine'].mean())
notSepsis['wbcx1000'] = notSepsis['wbcx1000'].fillna(notSepsis['wbcx1000'].mean())
notSepsis['lactate'] = notSepsis['lactate'].fillna(notSepsis['lactate'].mean())
notSepsis['urineoutputbyweight'] = notSepsis['urineoutputbyweight'].fillna(notSepsis['urineoutputbyweight'].mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [15]:
notSepsis[cols].isnull().sum().sum()

0

In [16]:
finalData2 = notSepsis.append(sepsisOnly)
finalData2

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
0,141227,-1893.0,38.308853,112.000000,49.0,158.000000,1.400000,48.200000,4.300000,2.433090,0.0
1,141227,-1773.0,38.308853,112.000000,49.0,158.000000,1.400000,48.200000,4.300000,2.433090,0.0
2,141227,-1663.0,38.308853,112.000000,49.0,158.000000,1.400000,48.200000,4.300000,2.433090,0.0
3,141227,-1566.0,38.308853,112.000000,49.0,158.000000,1.400000,48.200000,4.300000,2.433090,0.0
4,141227,-1351.0,38.308853,112.000000,49.0,158.000000,1.400000,47.950000,4.300000,2.433090,0.0
...,...,...,...,...,...,...,...,...,...,...,...
38719390,3353251,5521.0,37.700000,84.000000,17.0,125.666667,3.292857,11.952381,2.638305,0.859729,1.0
38719552,3353251,6303.0,37.700000,94.666667,20.0,151.333333,3.325918,12.558503,2.638305,1.884532,1.0
38720151,3353251,9169.0,37.700000,93.000000,25.0,150.000000,3.622759,14.797931,2.638305,2.724578,1.0
38720606,3353251,11304.0,37.700000,75.000000,17.0,130.000000,3.972642,17.783946,2.638305,2.521008,1.0


In [17]:
finalData2 = finalData2.sort_values(['patientunitstayid','observationoffset'])
finalData2

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
0,141227,-1893.0,38.308853,112.0,49.0,158.000000,1.40,48.20,4.300000,2.433090,0.0
1,141227,-1773.0,38.308853,112.0,49.0,158.000000,1.40,48.20,4.300000,2.433090,0.0
2,141227,-1663.0,38.308853,112.0,49.0,158.000000,1.40,48.20,4.300000,2.433090,0.0
3,141227,-1566.0,38.308853,112.0,49.0,158.000000,1.40,48.20,4.300000,2.433090,0.0
4,141227,-1351.0,38.308853,112.0,49.0,158.000000,1.40,47.95,4.300000,2.433090,0.0
...,...,...,...,...,...,...,...,...,...,...,...
38721752,3353254,5326.0,38.308853,82.0,12.0,118.646024,1.65,11.32,2.348331,4.767580,0.0
38721753,3353254,5491.0,38.308853,82.0,12.0,118.646024,1.66,11.51,2.348331,2.383790,0.0
38721754,3353254,5558.0,38.308853,82.0,12.0,118.646024,1.67,11.70,2.348331,4.767580,0.0
38721755,3353254,5926.0,38.308853,82.0,12.0,118.646024,1.67,11.70,2.348331,7.151371,0.0


In [18]:
finalData2 = finalData2.astype({'diagnosis': int})
finalData2.head()

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
0,141227,-1893.0,38.308853,112.0,49.0,158.0,1.4,48.20,4.3,2.43309,0
1,141227,-1773.0,38.308853,112.0,49.0,158.0,1.4,48.20,4.3,2.43309,0
2,141227,-1663.0,38.308853,112.0,49.0,158.0,1.4,48.20,4.3,2.43309,0
3,141227,-1566.0,38.308853,112.0,49.0,158.0,1.4,48.20,4.3,2.43309,0
4,141227,-1351.0,38.308853,112.0,49.0,158.0,1.4,47.95,4.3,2.43309,0


In [19]:
path = "../eICU/training/"
finalData.to_csv(path + "finalData2.csv", sep=',', index=False, encoding='utf-8')

In [ ]:
finalData = pd.read_csv('../eICU' + database_type + '/finalData2.csv')
finalData = finalData.sort_values(by=['patientunitstayid', 'observationoffset'])
finalData